In [1]:
from src_components.data_extraction.extract import (
    get_existing_citibike_files,
    find_all_downloadable_files,
    find_files_of_interest,
    download_files,
    unzip_file_recursive
)
from src_components.data_extraction.organize import (cleanup_and_organize)
from src_constants.extract_constants import (
    CITIBIKE_URL,
    RAW_DATA_FOLDER,
    RE_PATTERN,
    YEAR_FLOOR,
)

In [ ]:
existing_files = get_existing_citibike_files(RAW_DATA_FOLDER)
downloadable_files = find_all_downloadable_files(CITIBIKE_URL)

In [ ]:
files_of_interest = find_files_of_interest(downloadable_files, re_pattern=RE_PATTERN, year_floor=YEAR_FLOOR)
print(files_of_interest)

In [ ]:
download_files(RAW_DATA_FOLDER, CITIBIKE_URL, download_option= "missing")

In [ ]:
unzip_file_recursive(
    RAW_DATA_FOLDER
)

In [2]:
cleanup_and_organize()

Duplicate subfolder found at /Users/ashwin/Desktop/cpz_citibike/data/raw_citibike_data/2021-citibike-tripdata/2021-citibike-tripdata
Moving 202108-citibike-tripdata to /Users/ashwin/Desktop/cpz_citibike/data/raw_citibike_data/2021-citibike-tripdata
Moving 202107-citibike-tripdata.zip to /Users/ashwin/Desktop/cpz_citibike/data/raw_citibike_data/2021-citibike-tripdata
Moving 202111-citibike-tripdata.zip to /Users/ashwin/Desktop/cpz_citibike/data/raw_citibike_data/2021-citibike-tripdata
Moving 202111-citibike-tripdata to /Users/ashwin/Desktop/cpz_citibike/data/raw_citibike_data/2021-citibike-tripdata
Moving .DS_Store to /Users/ashwin/Desktop/cpz_citibike/data/raw_citibike_data/2021-citibike-tripdata
Moving 202109-citibike-tripdata to /Users/ashwin/Desktop/cpz_citibike/data/raw_citibike_data/2021-citibike-tripdata
Moving 202110-citibike-tripdata to /Users/ashwin/Desktop/cpz_citibike/data/raw_citibike_data/2021-citibike-tripdata
Moving 202105-citibike-tripdata.zip to /Users/ashwin/Desktop/c